In [1]:
import pandas as pd
import numpy as np
import numpy.random as npr
from datetime import datetime, timedelta
import calendar
from pyproj import Transformer
from concurrent.futures import ProcessPoolExecutor
import concurrent.futures
import multiprocessing
from tqdm import tqdm
from itertools import islice
from functools import partial
import copy

import nomad.city_gen as cg
from nomad.city_gen import City, Building, Street
import nomad.traj_gen
from nomad.traj_gen import Agent, Population
from nomad.constants import DEFAULT_SPEEDS, FAST_SPEEDS, SLOW_SPEEDS, DEFAULT_STILL_PROBS
from nomad.constants import FAST_STILL_PROBS, SLOW_STILL_PROBS, ALLOWED_BUILDINGS

### Generate N agents

The following code maps our Garden City coordinates to a location in the Atlantic Ocean (Atlantis?).

In [ ]:
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)


def garden_city_to_lat_long(agent, 
                            sparse_traj=True, 
                            full_traj=True):
    if sparse_traj:
        df = agent.sparse_traj
        df['x'] = 15*df['x'] + 4265699
        df['y'] = 15*df['y'] - 4392976

        df['longitude'], df['latitude'] = transformer.transform(
            df['x'].values, df['y'].values)

        df['date'] = df['local_timestamp'].dt.date  # for partitioning purposes

        df = df[['identifier', 'unix_timestamp', 'tz_offset', 'longitude', 'latitude', 'date']]
        df = df.rename(columns={'identifier': 'uid', 'unix_timestamp': 'timestamp'})
        df = df.reset_index(drop=True)

        agent.sparse_traj = df

    if full_traj:
        df = agent.trajectory
        df['x'] = 15*df['x'] + 4265699
        df['y'] = 15*df['y'] - 4392976

        df['longitude'], df['latitude'] = transformer.transform(
            df['x'].values, df['y'].values)

        df['date'] = df['local_timestamp'].dt.date  # for partitioning purposes

        df = df[['identifier', 'unix_timestamp', 'tz_offset', 'longitude', 'latitude', 'date']]
        df = df.rename(columns={'identifier': 'uid', 'unix_timestamp': 'timestamp'})
        df = df.reset_index(drop=True)

        agent.trajectory = df
    return None

Initiate $N$ empty agents. 

In [27]:
N = 100  # can modify
npr.seed(100)

city = cg.load('garden-city.pkl')
population_n = Population(city)
population_n.generate_agents(N,
                             start_time=pd.Timestamp('2024-01-01 08:00', tz='America/New_York'),
                             dt=1, seed=123,
                             name_count=2)

## Simple trajectory generation

For simple trajectory generation tasks that don't require too much computation power and can be done on a personal laptop, the following code generates a trajectory for each agent and saves it to a csv.

In [28]:
# Without using parallel processing (e.g., on personal laptop)

for i, agent_id in enumerate(population_n.roster):
    agent = population_n.roster[agent_id]

    beta_duration = npr.uniform(15, 180)
    beta_start = max(npr.uniform(60, 1200), beta_duration*3)
    beta_ping = npr.uniform(1.5, 30)
    param = (beta_start, beta_duration, beta_ping)

    population_n.generate_trajectory(agent, 
                                     T = pd.Timestamp('2024-01-15 08:00', tz='America/New_York'),
                                     seed=i)
    
    agent.sample_traj_hier_nhpp(*param, 
                                seed=i)
    
    garden_city_to_lat_long(agent,
                            sparse_traj=True,
                            full_traj=False)

In [ ]:
roster = population_n.roster
sparse_trajs = pd.concat([agent.sparse_traj for _, agent in roster.items()]).reset_index(drop=True)
sparse_trajs.to_csv('../nomad/data/gc_sample.csv', index=False)

## Parallelized Trajectory Generation

For larger trajectory generation tasks that require a lot of compute power, we can parallelize the trajectory generation using the following code. We generate ground-truth trajectories in agent-month "chunks", sparsify each chunk, then reset the ground-truth trajectory field to lessen the memory usage. 

In [ ]:
# Using parallel processing (e.g., using a cluster)
%%time

def generate_agent_trajectory(agent_id, agent, seed):
    
    beta_duration = npr.uniform(15, 180)
    beta_start = max(npr.uniform(60, 1200), beta_duration*3)
    beta_ping = npr.uniform(1.5, 30)
    
    param = (beta_start, beta_duration, beta_ping)
    
    for month in range(1,13):
        days = calendar.monthrange(2024, month)[1]
        population_n.generate_trajectory(agent, 
                                         T=datetime(2024, month, days, hour=23, minute=59), 
                                         seed=seed)
    
        agent.sample_traj_hier_nhpp(*param, 
                                    seed=seed,
                                    reset_traj=True)
    
    garden_city_to_lat_long(agent,
                            sparse_traj=True,
                            full_traj=False)
    
    return agent_id, copy.deepcopy(agent)

manager = multiprocessing.Manager()
shared_roster = manager.dict(population_n.roster)

start = 6001  # 12001  # can modify
end = 12001   # 18001  # can modify
roster = dict(population_n.roster)
batch = islice(roster.items(), start, end)

with ProcessPoolExecutor() as executor:
    with tqdm(total=(end-start), desc="Processing agents") as pbar:
        futures = [
            executor.submit(generate_agent_trajectory, agent_id, agent, i+15000)
            for i, (agent_id, agent) in enumerate(batch, start=start)
        ]
        results = []
        for future in futures:
            results.append(future.result())
            pbar.update(1)

for agent_id, agent in results:
    population_n.roster[agent_id] = agent

Processing agents:  10%|▉         | 592/6000 [27:37<4:19:55,  2.88s/it] 

This code saves the generated trajectories in a parquet file, using the date as the partition column.

In [ ]:
partition_cols = {
    'sparse_traj': ['date'],
    'diaries': ['identifier']
}

roster = dict(islice(population_n.roster.items(), start, end))

population_n.save_pop(bucket="synthetic-raw-data",
                      prefix=f"agents-{start+15000}-{end+15000-1}/",
                      save_full_traj=False,
                      save_sparse_traj=True,
                      save_homes=True,
                      save_diaries=True,
                      partition_cols=partition_cols,
                      roster=roster)

In [ ]:
# Load the Parquet files

s3_path = "s3://synthetic-raw-data/agents-1-1001/sparse_trajectories.parquet/"
df1 = pd.read_parquet(s3_path)
s3_path = "s3://synthetic-raw-data/agents-1001-2000/sparse_trajectories.parquet/"
df2 = pd.read_parquet(s3_path)